### Input types

In [1]:
from supabase import create_client, Client
import pandas as pd
import os as os
from dotenv import load_dotenv

load_dotenv()

# Initialize Supabase client
url: str = os.environ.get("SUPABASE_PUBLIC_URL")
key: str = os.environ.get("SUPABASE_SERVICE_ROLE_KEY")
supabase: Client = create_client(url, key)

In [7]:
# Function to read the file and return a list of entries
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read().splitlines()

# Function to synchronize file entries into Supabase
def synchronize_entries(supabase, table_name, column_name, entries):
    # Fetch existing entries from the table
    existing_entries_result = supabase.table(table_name).select(column_name).execute()
    existing_entries = [entry[column_name] for entry in existing_entries_result.data]

    # Entries to be added (present in file but not in the table)
    entries_to_add = [entry for entry in entries if entry not in existing_entries]

    # Entries to be deleted (present in the table but not in the file)
    entries_to_delete = [entry for entry in existing_entries if entry not in entries]

    # Insert new entries
    if entries_to_add:
        supabase.table(table_name).insert([{column_name: entry} for entry in entries_to_add]).execute()
        print(f"Inserted {len(entries_to_add)} new entries into table {table_name}.")

    # Delete entries not in file
    if entries_to_delete:
        for entry in entries_to_delete:
            supabase.table(table_name).delete().eq(column_name, entry).execute()
        print(f"Deleted {len(entries_to_delete)} entries from table {table_name}.")

    if not entries_to_add and not entries_to_delete:
        print(f"Table {table_name} is already synchronized with the file.")

In [8]:
# Synchronize entries in Supabase
synchronize_entries(supabase, '_acceptance', 'acceptance', read_file('tables/_acceptance.txt'))
synchronize_entries(supabase, '_actions', 'action', read_file('tables/_actions.txt'))
synchronize_entries(supabase, '_place_types', 'place_type', read_file('tables/_place_types.txt'))
synchronize_entries(supabase, '_sex', 'sex', read_file('tables/_sex.txt'))

2023-11-21 11:59:32,623:INFO - HTTP Request: GET https://supabase.cloud-atlas.xyz/rest/v1/_acceptance?select=acceptance "HTTP/1.1 200 OK"
2023-11-21 11:59:32,635:INFO - HTTP Request: GET https://supabase.cloud-atlas.xyz/rest/v1/_actions?select=action "HTTP/1.1 200 OK"
2023-11-21 11:59:32,648:INFO - HTTP Request: GET https://supabase.cloud-atlas.xyz/rest/v1/_place_types?select=place_type "HTTP/1.1 200 OK"
2023-11-21 11:59:32,660:INFO - HTTP Request: GET https://supabase.cloud-atlas.xyz/rest/v1/_sex?select=sex "HTTP/1.1 200 OK"


Table _acceptance is already synchronized with the file.
Table _actions is already synchronized with the file.
Table _place_types is already synchronized with the file.
Table _sex is already synchronized with the file.
